In [2]:
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

sys.path.append(os.path.abspath(os.path.join('/workspaces/ECG-ML-Challenge/notebooks/', "..")))
from src.utilities.dataloader import ECGDataLoader
from src.utilities.model import ECGGRUModel
from src.dataset import download_wfdb_dataset
from src import config

In [11]:
download_wfdb_dataset('afdb', os.path.join(config.RAW_DATA_DIR, 'afdb'))
!python ../src/dataset.py afdb 250

2024-11-08 19:22:42.595 | INFO     | src.dataset:download_wfdb_dataset:37 - afdb already exists in /workspaces/ECG-ML-Challenge/data/raw/afdb.
2024-11-08 19:22:43.633 | INFO     | config:<module>:8 - PROJ_ROOT path is: /workspaces/ECG-ML-Challenge
2024-11-08 19:22:43.642 | INFO     | __main__:download_wfdb_dataset:36 - afdb already exists in /workspaces/ECG-ML-Challenge/data/raw/afdb.
2024-11-08 19:22:43.642 | INFO     | __main__:main:146 - Processing 25 ECG records...
Processing files: 100%|█████████████████████████| 25/25 [00:44<00:00,  1.78s/it]
2024-11-08 19:23:28.123 | INFO     | __main__:main:179 - All ECG data saved to /workspaces/ECG-ML-Challenge/data/processed/afdb


In [4]:
# Set parameters
input_size = 1  # Number of input features
hidden_size = 128  # Number of features in the hidden state
num_layers = 2  # Number of recurrent layers
output_size = 1  # Number of output features
batch_size = 32
num_epochs = 10
learning_rate = 0.001

# Load data
processed_data_dir = os.path.join(config.PROCESSED_DATA_DIR, 'afdb')
train_loader = ECGDataLoader(processed_data_dir, batch_size=batch_size, shuffle=True)

# Initialize model, loss function, and optimizer
model = ECGGRUModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

# Save the model
torch.save(model.state_dict(), 'ecg_gru_model.pth')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy: {100 * correct / total}%')

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/worker.py", line 351, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/fetch.py", line 55, in fetch
    return self.collate_fn(data)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 398, in default_collate
    return collate(batch, collate_fn_map=default_collate_fn_map)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 211, in collate
    return [
           ^
  File "/opt/conda/lib/python3.11/site-packages/torch/utils/data/_utils/collate.py", line 212, in <listcomp>
    collate(samples, collate